In [1]:
import math
import numpy as np
import os

import nemo
from nemo.utils.lr_policies import WarmupAnnealing

import nemo_nlp
from nemo_nlp import NemoBertTokenizer, TokenClassifier, TokenClassificationLoss
from nemo_nlp.data.datasets import utils
from nemo_nlp.utils.callbacks.punctuation_capitalization import eval_iter_callback, eval_epochs_done_callback

DATA_DIR = "PATH_TO_WHERE_THE_DATA_IS"
WORK_DIR = "PATH_TO_WHERE_TO_STORE_CHECKPOINTS_AND_LOGS"
PRETRAINED_BERT_MODEL = "bert-base-uncased"

# model parameters
BATCHES_PER_STEP = 1
BATCH_SIZE = 64
CLASSIFICATION_DROPOUT = 0.1
MAX_SEQ_LENGTH = 128
NUM_EPOCHS = 5
LEARNING_RATE = 0.00002
LR_WARMUP_PROPORTION = 0.1
OPTIMIZER = "adam"
STEP_FREQ=200 # determines how often loss will be printed and checkpoint saved
PUNCT_NUM_FC_LAYERS = 3

# Download and preprocess the data

In this notebook we're going to use a subset of English examples from the [Tatoeba collection of sentences](https://tatoeba.org/eng), set NUM_SAMPLES=-1 and consider including other datasets to training set to improve the performance of the model. Use [NeMo/scripts/get_tatoeba_data.py](https://github.com/NVIDIA/NeMo/blob/master/scripts/get_tatoeba_data.py) to download and preprocess the Tatoeba data.

In [2]:
# This should take about a minute since the data is already downloaded in the previous step
! python ../../scripts/get_tatoeba_data.py --data_dir $DATA_DIR

Processing English sentences...
Splitting the tatoeba dataset into train and dev sets and creating labels and text files
Creating text and label files for training
Processing of the tatoeba is complete


After the previous step, you should have a `DATA_DIR` folder with the following files:
- labels_train.txt
- labels_dev.txt
- text_train.txt
- text_dev.txt

The format of the data described in NeMo docs.

# Define Neural Modules

In [3]:
# Instantiate neural factory with supported backend
nf = nemo.core.NeuralModuleFactory(
    backend=nemo.core.Backend.PyTorch,

    # If you're training with multiple GPUs, you should handle this value with
    # something like argparse. See examples/nlp/token_classification.py for an example.
    local_rank=None,

    # If you're training with mixed precision, this should be set to mxprO1 or mxprO2.
    # See https://nvidia.github.io/apex/amp.html#opt-levels for more details.
    optimization_level="O0",
    
    # Define path to the directory you want to store your results
    log_dir=WORK_DIR,

    # If you're training with multiple GPUs, this should be set to
    # nemo.core.DeviceType.AllGpu
    placement=nemo.core.DeviceType.GPU)

In [4]:
# If you're using a standard BERT model, you should do it like this. To see the full
# list of BERT model names, check out nemo_nlp.huggingface.BERT.list_pretrained_models()

tokenizer = NemoBertTokenizer(pretrained_model=PRETRAINED_BERT_MODEL)
bert_model = nemo_nlp.huggingface.BERT(pretrained_model_name=PRETRAINED_BERT_MODEL)

2019-12-20 16:15:58,893 - INFO - loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/ebakhturina/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2019-12-20 16:15:59,351 - INFO - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/ebakhturina/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
2019-12-20 16:15:59,354 - INFO - Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads":

# Describe training DAG

In [5]:
train_data_layer = nemo_nlp.BertPunctuationCapitalizationDataLayer(
                                                             tokenizer=tokenizer,
                                                             text_file=os.path.join(DATA_DIR, 'text_train.txt'),
                                                             label_file=os.path.join(DATA_DIR, 'labels_train.txt'),
                                                             max_seq_length=MAX_SEQ_LENGTH,
                                                             batch_size=BATCH_SIZE)

punct_label_ids = train_data_layer.dataset.punct_label_ids
capit_label_ids = train_data_layer.dataset.capit_label_ids

hidden_size = bert_model.local_parameters["hidden_size"]


# Define classifier for Punctuation and Capitalization tasks
punct_classifier = nemo_nlp.TokenClassifier(hidden_size=hidden_size,
                                            num_classes=len(punct_label_ids),
                                            dropout=CLASSIFICATION_DROPOUT,
                                            num_layers=PUNCT_NUM_FC_LAYERS,
                                            name='Punctuation')

capit_classifier = nemo_nlp.TokenClassifier(hidden_size=hidden_size,
                                            num_classes=len(capit_label_ids),
                                            dropout=CLASSIFICATION_DROPOUT,
                                            name='Capitalization')


# If you don't want to use weighted loss for Punctuation task, use class_weights=None
punct_label_freqs = train_data_layer.dataset.punct_label_frequencies
class_weights = utils.calc_class_weights(punct_label_freqs)

# define loss
punct_loss = nemo_nlp.TokenClassificationLoss(num_classes=len(punct_label_ids),
                                              class_weights=class_weights)
capit_loss = nemo_nlp.TokenClassificationLoss(num_classes=len(capit_label_ids))
task_loss = nemo_nlp.LossAggregatorNM(num_inputs=2)

2019-12-20 16:16:06,300 - INFO - Creating a new label to label_id dictionary. It's recommended to use label_ids generated during training for dev/test sets to avoid errors if some labels are not present in the dev/test sets. For training set label_ids should be None.
2019-12-20 16:20:05,998 - INFO - Max length: 128
2019-12-20 16:20:06,054 - INFO - Min: 3 |                  Max: 135 |                  Mean: 10.321342371917975 |                  Median: 10.0
2019-12-20 16:20:06,061 - INFO - 75 percentile: 12.0
2019-12-20 16:20:06,069 - INFO - 99 percentile: 23.0
2019-12-20 16:20:41,264 - INFO - 3 are longer than 128
2019-12-20 16:20:41,268 - INFO - *** Example ***
2019-12-20 16:20:41,269 - INFO - i: 0
2019-12-20 16:20:41,269 - INFO - subtokens: [CLS] let ' s try something [SEP]
2019-12-20 16:20:41,270 - INFO - loss_mask: 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

2019-12-20 16:20:41,293 - INFO - punct_labels: 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
2019-12-20 16:20:41,294 - INFO - capit_labels: 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
2019-12-20 16:20:41,592 - INFO - Three most popular labels
2019-12-20 16:20:46,366 - INFO - 0 item: 0, 121383875 out of 122506496, 0.9908362328802548.
2019-12-20 16:20:46,367 - INFO - 1 item: 2, 900824 out of 122506496, 0.007353275372434128.
2019-12-20 16:20:46,368 - INFO - 2 item: 3, 124860 out of 122506496, 0.0010192112588054106.
2019-12-20 16:20:46,368 - INFO - Labels: {'O': 0, ',': 1, '.

In [6]:
input_ids, input_type_ids, input_mask, loss_mask, subtokens_mask, punct_labels, capit_labels = train_data_layer()

hidden_states = bert_model(input_ids=input_ids,
                      token_type_ids=input_type_ids,
                      attention_mask=input_mask)

punct_logits = punct_classifier(hidden_states=hidden_states)
capit_logits = capit_classifier(hidden_states=hidden_states)

punct_loss = punct_loss(logits=punct_logits,
                        labels=punct_labels,
                        loss_mask=loss_mask)
capit_loss = capit_loss(logits=capit_logits,
                        labels=capit_labels,
                        loss_mask=loss_mask)
task_loss = task_loss(loss_1=punct_loss,
                      loss_2=capit_loss)

# Describe evaluation DAG

In [7]:
# Note that you need to specify punct_label_ids and capit_label_ids  - mapping form labels to label_ids generated
# during creation of the train_data_layer to make sure that the mapping is correct in case some of the labels from
# the train set are missing in the dev set.

eval_data_layer = nemo_nlp.BertPunctuationCapitalizationDataLayer(
                                                            tokenizer=tokenizer,
                                                            text_file=os.path.join(DATA_DIR, 'text_dev.txt'),
                                                            label_file=os.path.join(DATA_DIR, 'labels_dev.txt'),
                                                            max_seq_length=MAX_SEQ_LENGTH,
                                                            batch_size=BATCH_SIZE,
                                                            punct_label_ids=punct_label_ids,
                                                            capit_label_ids=capit_label_ids)

eval_input_ids, eval_input_type_ids, eval_input_mask, _, eval_subtokens_mask, eval_punct_labels, eval_capit_labels\
    = eval_data_layer()

hidden_states = bert_model(input_ids=eval_input_ids,
                           token_type_ids=eval_input_type_ids,
                           attention_mask=eval_input_mask)

eval_punct_logits = punct_classifier(hidden_states=hidden_states)
eval_capit_logits = capit_classifier(hidden_states=hidden_states)

2019-12-20 16:20:51,571 - INFO - Using the provided label_ids dictionary.
2019-12-20 16:21:48,011 - INFO - Max length: 128
2019-12-20 16:21:48,025 - INFO - Min: 3 |                  Max: 167 |                  Mean: 10.566364358256363 |                  Median: 10.0
2019-12-20 16:21:48,027 - INFO - 75 percentile: 12.0
2019-12-20 16:21:48,029 - INFO - 99 percentile: 22.0
2019-12-20 16:22:01,402 - INFO - 1 are longer than 128
2019-12-20 16:22:01,406 - INFO - *** Example ***
2019-12-20 16:22:01,407 - INFO - i: 0
2019-12-20 16:22:01,408 - INFO - subtokens: [CLS] i thought that you knew tom and mary were married [SEP]
2019-12-20 16:22:01,408 - INFO - loss_mask: 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
2019-12-20 16:22:01,409 - INFO - input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 

2019-12-20 16:22:01,425 - INFO - punct_labels: 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
2019-12-20 16:22:01,426 - INFO - capit_labels: 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
2019-12-20 16:22:01,512 - INFO - Three most popular labels
2019-12-20 16:22:02,704 - INFO - 0 item: 0, 30348057 out of 30626560, 0.9909064877021775.
2019-12-20 16:22:02,706 - INFO - 1 item: 2, 237578 out of 30626560, 0.007757253834580181.
2019-12-20 16:22:02,706 - INFO - 2 item: 3, 20590 out of 30626560, 0.0006722922848664689.
2019-12-20 16:22:02,707 - INFO - Labels: {'O': 0, ',': 1, '.': 2,

# Create callbacks

In [8]:
callback_train = nemo.core.SimpleLossLoggerCallback(
    tensors=[task_loss, punct_loss, capit_loss, punct_logits, capit_logits],
    print_func=lambda x: print("Loss: {:.3f}".format(x[0].item())),
    step_freq=STEP_FREQ)

train_data_size = len(train_data_layer)

# If you're training on multiple GPUs, this should be
# train_data_size / (batch_size * batches_per_step * num_gpus)
steps_per_epoch = int(train_data_size / (BATCHES_PER_STEP * BATCH_SIZE))
nf.logger.info(f'steps_per_epoch: {steps_per_epoch}')

# Callback to evaluate the model
callback_eval = nemo.core.EvaluatorCallback(
    eval_tensors=[eval_punct_logits,
                  eval_capit_logits,
                  eval_punct_labels,
                  eval_capit_labels,
                  eval_subtokens_mask],
    user_iter_callback=lambda x, y: eval_iter_callback(x, y),
    user_epochs_done_callback=lambda x: eval_epochs_done_callback(x,
                                                                  punct_label_ids,
                                                                  capit_label_ids),
    eval_step=steps_per_epoch)

# Callback to store checkpoints
ckpt_callback = nemo.core.CheckpointCallback(folder=nf.checkpoint_dir,
                                             step_freq=STEP_FREQ)

2019-12-20 16:22:04,018 - INFO - steps_per_epoch: 14954


# Training

In [9]:
lr_policy = WarmupAnnealing(NUM_EPOCHS * steps_per_epoch,
                            warmup_ratio=LR_WARMUP_PROPORTION)

import time
start = time.time()
nf.train(tensors_to_optimize=[task_loss],
         callbacks=[callback_train, callback_eval, ckpt_callback],
         lr_policy=lr_policy,
         batches_per_step=BATCHES_PER_STEP,
         optimizer=OPTIMIZER,
         optimization_params={"num_epochs": NUM_EPOCHS,
                              "lr": LEARNING_RATE})

print (time.time() - start)

2019-12-20 16:22:04,036 - WARNING - Data Layer does not have any weights to return. This get_weights call returns None.
2019-12-20 16:22:04,038 - INFO - Starting .....
2019-12-20 16:22:04,040 - INFO - Found 3 modules with weights:
2019-12-20 16:22:04,041 - INFO - PunctuationTokenClassifier
2019-12-20 16:22:04,042 - INFO - BERT
2019-12-20 16:22:04,042 - INFO - CapitalizationTokenClassifier
2019-12-20 16:22:04,043 - INFO - Total model parameters: 111258630
2019-12-20 16:22:04,044 - INFO - Restoring checkpoint from folder PATH_TO_WHERE_TO_STORE_CHECKPOINTS_AND_LOGS/checkpoints ...
2019-12-20 16:22:04,965 - INFO - Starting epoch 0
2019-12-20 16:22:08,378 - INFO - Step: 25200


Loss: 0.066


2019-12-20 16:22:08,381 - INFO - Step time: 0.5306496620178223 seconds
2019-12-20 16:22:11,171 - INFO - Saved checkpoint: PATH_TO_WHERE_TO_STORE_CHECKPOINTS_AND_LOGS/checkpoints/trainer-STEP-25200.pt
2019-12-20 16:23:54,771 - INFO - Step: 25400


Loss: 0.007


2019-12-20 16:23:54,773 - INFO - Step time: 0.5250601768493652 seconds
2019-12-20 16:23:55,539 - INFO - Saved checkpoint: PATH_TO_WHERE_TO_STORE_CHECKPOINTS_AND_LOGS/checkpoints/trainer-STEP-25400.pt
2019-12-20 16:25:41,412 - INFO - Step: 25600


...

Loss: 0.002


2019-12-21 04:51:48,384 - INFO - Step time: 0.5311088562011719 seconds
2019-12-21 04:51:49,222 - INFO - Saved checkpoint: PATH_TO_WHERE_TO_STORE_CHECKPOINTS_AND_LOGS/checkpoints/trainer-STEP-99600.pt
2019-12-21 04:53:35,968 - INFO - Step: 99800


Loss: 0.005


2019-12-21 04:53:35,969 - INFO - Step time: 0.5301709175109863 seconds
2019-12-21 04:53:36,804 - INFO - Saved checkpoint: PATH_TO_WHERE_TO_STORE_CHECKPOINTS_AND_LOGS/checkpoints/trainer-STEP-99800.pt
2019-12-21 04:55:09,774 - INFO - Finished epoch 4 in 9041.044075727463
2019-12-21 04:55:09,787 - INFO - Done in 45185.74810361862
2019-12-21 04:55:09,791 - INFO - Final Evaluation ..............................
2019-12-21 05:11:52,504 - INFO - Accuracy for task punct: 0.9876787113756859
2019-12-21 05:11:52,506 - INFO - Sampled preds: [2 0 0 0 0 1 0 0 0 0 2 0 1 0 0 0 0 2 0 0]
2019-12-21 05:11:52,506 - INFO - Sampled labels: [2 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 0]
2019-12-21 05:11:54,357 - INFO -               precision    recall  f1-score   support

           O       1.00      0.99      0.99   1544420
           ,       0.46      0.98      0.62     17627
           .       1.00      1.00      1.00    221292
           ?       0.98      0.99      0.99     19638

    accuracy              

46200.79797697067


# Inference

In [10]:
# Define the list of queiries for inference
queries = ['we bought four shirts from the nvidia gear store in santa clara', 
           'nvidia is a company',
           'can i help you',
           'how are you',
           'okay',
           'we bought four shirts one mug and ten thousand titan rtx graphics cards the more you buy the more you save']

In [11]:
infer_data_layer = nemo_nlp.BertTokenClassificationInferDataLayer(queries=queries,
                                                                  tokenizer=tokenizer,
                                                                  max_seq_length=MAX_SEQ_LENGTH,
                                                                  batch_size=1)
input_ids, input_type_ids, input_mask, _, subtokens_mask = infer_data_layer()

hidden_states = bert_model(input_ids=input_ids,
                                      token_type_ids=input_type_ids,
                                      attention_mask=input_mask)
punct_logits = punct_classifier(hidden_states=hidden_states)
capit_logits = capit_classifier(hidden_states=hidden_states)

evaluated_tensors = nf.infer(tensors=[punct_logits, capit_logits, subtokens_mask],
                             checkpoint_dir=WORK_DIR + '/checkpoints')

2019-12-21 05:12:04,879 - INFO - Max length: 24
2019-12-21 05:12:04,880 - INFO - Min: 3 |                  Max: 24 |                  Mean: 10.333333333333334 |                  Median: 7.0
2019-12-21 05:12:04,882 - INFO - 75 percentile: 14.0
2019-12-21 05:12:04,883 - INFO - 99 percentile: 23.6
2019-12-21 05:12:04,884 - INFO - 0 are longer than 24
2019-12-21 05:12:04,885 - INFO - *** Example ***
2019-12-21 05:12:04,885 - INFO - i: 0
2019-12-21 05:12:04,886 - INFO - subtokens: [CLS] we bought four shirts from the n ##vid ##ia gear store in santa clara [SEP]
2019-12-21 05:12:04,887 - INFO - loss_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0
2019-12-21 05:12:04,887 - INFO - input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0
2019-12-21 05:12:04,888 - INFO - subtokens_mask: 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0
2019-12-21 05:12:04,889 - INFO - *** Example ***
2019-12-21 05:12:04,889 - INFO - i: 1
2019-12-21 05:12:04,890 - INFO - subtokens: [CLS] n ##vid ##ia is a 

In [12]:
# helper functions
def concatenate(lists):
    return np.concatenate([t.cpu() for t in lists])

punct_ids_to_labels = {punct_label_ids[k]: k for k in punct_label_ids}
capit_ids_to_labels = {capit_label_ids[k]: k for k in capit_label_ids}

punct_logits, capit_logits, subtokens_mask = [concatenate(tensors) for tensors in evaluated_tensors]
punct_preds = np.argmax(punct_logits, axis=2)
capit_preds = np.argmax(capit_logits, axis=2)

for i, query in enumerate(queries):
    nf.logger.info(f'Query: {query}')

    punct_pred = punct_preds[i][subtokens_mask[i] > 0.5]
    capit_pred = capit_preds[i][subtokens_mask[i] > 0.5]
    words = query.strip().split()
    if len(punct_pred) != len(words) or len(capit_pred) != len(words):
        raise ValueError('Pred and words must be of the same length')

    output = ''
    for j, w in enumerate(words):
        punct_label = punct_ids_to_labels[punct_pred[j]]
        capit_label = capit_ids_to_labels[capit_pred[j]]

        if capit_label != 'O':
            w = w.capitalize()
        output += w
        if punct_label != 'O':
            output += punct_label
        output += ' '
    nf.logger.info(f'Combined: {output.strip()}\n')

2019-12-21 05:12:08,849 - INFO - Query: we bought four shirts from the nvidia gear store in santa clara
2019-12-21 05:12:08,850 - INFO - Combined: We bought four shirts from the Nvidia gear store in Santa Clara.

2019-12-21 05:12:08,850 - INFO - Query: nvidia is a company
2019-12-21 05:12:08,859 - INFO - Combined: Nvidia is a company.

2019-12-21 05:12:08,860 - INFO - Query: can i help you
2019-12-21 05:12:08,861 - INFO - Combined: Can I help you?

2019-12-21 05:12:08,862 - INFO - Query: how are you
2019-12-21 05:12:08,863 - INFO - Combined: How are you?

2019-12-21 05:12:08,864 - INFO - Query: okay
2019-12-21 05:12:08,865 - INFO - Combined: Okay.

2019-12-21 05:12:08,867 - INFO - Query: we bought four shirts one mug and ten thousand titan rtx graphics cards the more you buy the more you save
2019-12-21 05:12:08,867 - INFO - Combined: We bought four shirts, one mug, and ten thousand Titan Rtx, graphics cards. The more you buy, the more you save.

